## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
0,0,Georgetown,5.41,100.34,91.4,62,20,5.82,MY,few clouds,2020-08-24 06:19:08
1,1,Barrow,71.29,-156.79,39.2,93,90,8.05,US,mist,2020-08-24 06:19:08
2,2,Hermanus,-34.42,19.23,54.0,95,100,8.37,ZA,overcast clouds,2020-08-24 06:19:08
3,3,Pisco,-13.70,-76.22,61.0,87,90,12.75,PE,overcast clouds,2020-08-24 06:19:08
4,4,Kavaratti,10.57,72.64,86.0,70,20,9.17,IN,few clouds,2020-08-24 06:19:09


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
0,0,Georgetown,5.41,100.34,91.40,62,20,5.82,MY,few clouds,2020-08-24 06:19:08
4,4,Kavaratti,10.57,72.64,86.00,70,20,9.17,IN,few clouds,2020-08-24 06:19:09
9,9,Saint-Philippe,-21.36,55.77,77.00,57,37,6.93,RE,scattered clouds,2020-08-24 06:19:10
10,10,Sola,-13.88,167.55,81.45,82,37,9.24,VU,light rain,2020-08-24 06:19:11
12,12,Sao Filipe,14.90,-24.50,78.78,86,80,12.46,CV,broken clouds,2020-08-24 06:19:11
14,14,Hithadhoo,-0.60,73.08,83.14,76,98,10.54,MV,moderate rain,2020-08-24 06:19:12
16,16,Victoria,22.29,114.16,95.00,59,0,8.01,HK,clear sky,2020-08-24 06:16:51
21,21,Kavieng,-2.57,150.80,78.10,83,100,7.63,PG,light rain,2020-08-24 06:19:14
25,25,Carutapera,-1.20,-46.02,77.02,88,24,7.02,BR,few clouds,2020-08-24 06:19:15
26,26,Les Cayes,18.20,-73.75,80.55,85,100,13.27,HT,moderate rain,2020-08-24 06:19:16


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        241
City           241
Lat            241
Lng            241
Max Temp       241
Humidity       241
Cloudiness     241
Wind Speed     241
Country        241
Description    241
Date           241
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Georgetown,MY,91.40,few clouds,5.41,100.34,
4,Kavaratti,IN,86.00,few clouds,10.57,72.64,
9,Saint-Philippe,RE,77.00,scattered clouds,-21.36,55.77,
10,Sola,VU,81.45,light rain,-13.88,167.55,
12,Sao Filipe,CV,78.78,broken clouds,14.90,-24.50,
14,Hithadhoo,MV,83.14,moderate rain,-0.60,73.08,
16,Victoria,HK,95.00,clear sky,22.29,114.16,
21,Kavieng,PG,78.10,light rain,-2.57,150.80,
25,Carutapera,BR,77.02,few clouds,-1.20,-46.02,
26,Les Cayes,HT,80.55,moderate rain,18.20,-73.75,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Georgetown,MY,91.40,few clouds,5.41,100.34,Cititel Penang
4,Kavaratti,IN,86.00,few clouds,10.57,72.64,Bismillah Hotel And Lodging
9,Saint-Philippe,RE,77.00,scattered clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
10,Sola,VU,81.45,light rain,-13.88,167.55,Leumerus Bungalows
12,Sao Filipe,CV,78.78,broken clouds,14.90,-24.50,Tortuga B&B
...,...,...,...,...,...,...,...
645,Idil,TR,84.20,clear sky,37.34,41.89,ÇAKIR APARTMANI
646,Muncar,ID,84.42,few clouds,-8.43,114.33,Zaky Homestay
647,Katobu,ID,87.96,few clouds,-4.94,122.53,La Ode Husein
650,Oussouye,SN,80.60,few clouds,12.48,-16.55,Auberge du Routard


In [23]:
# 7. Replace Empty Rows with np.nan.
import numpy as np
hotel_df.replace('', np.nan, inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Georgetown,MY,91.40,few clouds,5.41,100.34,Cititel Penang
4,Kavaratti,IN,86.00,few clouds,10.57,72.64,Bismillah Hotel And Lodging
9,Saint-Philippe,RE,77.00,scattered clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
10,Sola,VU,81.45,light rain,-13.88,167.55,Leumerus Bungalows
12,Sao Filipe,CV,78.78,broken clouds,14.90,-24.50,Tortuga B&B
14,Hithadhoo,MV,83.14,moderate rain,-0.60,73.08,Scoop Guest House
16,Victoria,HK,95.00,clear sky,22.29,114.16,Mini Hotel Central
21,Kavieng,PG,78.10,light rain,-2.57,150.80,Nusa Island Retreat
25,Carutapera,BR,77.02,few clouds,-1.20,-46.02,Lidera
26,Les Cayes,HT,80.55,moderate rain,18.20,-73.75,Villa Orphee


In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

# Create a clean hotel DataFrame.
clean_hotel_df = hotel_df.copy()
clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Georgetown,MY,91.40,few clouds,5.41,100.34,Cititel Penang
4,Kavaratti,IN,86.00,few clouds,10.57,72.64,Bismillah Hotel And Lodging
9,Saint-Philippe,RE,77.00,scattered clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
10,Sola,VU,81.45,light rain,-13.88,167.55,Leumerus Bungalows
12,Sao Filipe,CV,78.78,broken clouds,14.90,-24.50,Tortuga B&B
14,Hithadhoo,MV,83.14,moderate rain,-0.60,73.08,Scoop Guest House
16,Victoria,HK,95.00,clear sky,22.29,114.16,Mini Hotel Central
21,Kavieng,PG,78.10,light rain,-2.57,150.80,Nusa Island Retreat
25,Carutapera,BR,77.02,few clouds,-1.20,-46.02,Lidera
26,Les Cayes,HT,80.55,moderate rain,18.20,-73.75,Villa Orphee


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))